In [2]:
from transformers import LlamaTokenizerFast
from datasets import load_dataset
import json


In [3]:
from transformers import LlamaTokenizerFast
from datasets import load_dataset
def preprocess_get_input_and_output(dataset_name, max_samples=50000):
    # Load the dataset
    dataset = load_dataset(dataset_name)

    # # Initialize lists to store input and output texts
    input_texts = []
    output_texts = []

    # Process each dataset individually based on its format
    if dataset_name == "casey-martin/MedInstruct":
        for example in dataset["train"].select(range(min(max_samples, len(dataset["train"])))):
            input_text = example["instruction"] + example["input"]
            output_text = example["output"]
            input_texts.append(input_text)
            output_texts.append(output_text)
    elif dataset_name == "TIGER-Lab/MathInstruct":
        for example in dataset["train"].select(range(min(max_samples, len(dataset["train"])))):
            input_text = example["instruction"]
            output_text = example["output"]
            input_texts.append(input_text)
            output_texts.append(output_text)
    elif dataset_name == "train_en_phy_chem.json":
        with open(dataset_name, 'r') as file:
            for i, line in enumerate(file):
                if i >= max_samples:
                    break
                example = json.loads(line)
                input_text = example["content"]
                output_text = example["summary"]
                input_texts.append(input_text)
                output_texts.append(output_text)
    elif dataset_name == "chemical_disease_interaction_extraction.json":
        with open(dataset_name, 'r') as file:
            data = json.load(file)
            # Process each example in the JSON data
            for example in data[:max_samples]:
                input_text = example["instruction"] + " " + example["input"]
                output_text = example["output"]
                input_texts.append(input_text)
                output_texts.append(output_text)
    elif dataset_name == 'osunlp/SMolInstruct':
        for example in dataset["train"].select(range(min(max_samples, len(dataset["train"])))):
            input_text = example["input"]
            output_text = example["output"]
            input_texts.append(input_text)
            output_texts.append(output_text)
    elif dataset_name == 'allenai/tulu-v2-sft-mixture':
        for example in dataset["train"].select(range(min(max_samples, len(dataset["train"])))):
            user_content = ""
            assistant_content = ""

            # Iterate over each message in the "messages" list
            for message in example["messages"]:
                if message["role"] == "user":
                    user_content += message["content"] + "\n"
                elif message["role"] == "assistant":
                    assistant_content += message["content"] + "\n"

            input_texts.append(user_content.strip())
            output_texts.append(assistant_content.strip())

    return input_texts, output_texts
    

    return input_texts, output_texts
    # Add more elif conditions for other datasets as needed

    return input_texts, output_texts

def compute_avg_token_length(tokenizer_model, input_texts, output_texts):
    # Load the tokenizer
    tokenizer = LlamaTokenizerFast.from_pretrained(tokenizer_model)

    # Initialize variables to store total token lengths
    total_input_tokens = 0
    total_output_tokens = 0

    # Tokenize and compute token lengths for the limited number of samples
    for i in range(len(input_texts)):
        input_tokens = tokenizer.encode(input_texts[i])
        output_tokens = tokenizer.encode(output_texts[i])
        total_input_tokens += len(input_tokens)
        total_output_tokens += len(output_tokens)

    # Compute average token lengths
    avg_input_length = total_input_tokens / len(input_texts)
    avg_output_length = total_output_tokens / len(output_texts)

    return avg_input_length, avg_output_length

# Example usage
dataset_names = ["allenai/tulu-v2-sft-mixture"]
tokenizer_model = "NousResearch/Llama-2-7b-hf"
max_samples = 3000000

for dataset_name in dataset_names:
    print(f"Dataset: {dataset_name}")
    input_texts, output_texts = preprocess_get_input_and_output(dataset_name, max_samples)
    avg_input_length, avg_output_length = compute_avg_token_length(tokenizer_model, input_texts, output_texts)

    print(f"Average input token length: {avg_input_length:.2f}")
    print(f"Average output token length: {avg_output_length:.2f}")
    print()

Dataset: allenai/tulu-v2-sft-mixture
Average input token length: 353.33
Average output token length: 696.89

